In [1]:
# 2020 Q2부터 시작할 수 있음..

# high_list = [
#     "4. 재무제표.csv",
#     "2. 연결재무제표.csv",
#     "1. 요약재무정보.csv",
#     "5. 재무제표 주석.csv",
#     "3. 연결재무제표 주석.csv",
#     "6. 배당에 관한 사항 등.csv",
#     "3. 자본금 변동사항.csv",
#     "VII. 주주에 관한 사항.csv",
#     "IX. 계열회사 등에 관한 사항.csv",
#     "XI. 그 밖에 투자자 보호를 위하여 필요한 사항.csv",
#     "III. 재무에 관한 사항.csv",
# ]
# medium_list = [
#     "4. 주식의 총수 등.csv",
#     "5. 의결권 현황.csv",
#     "1. 이사회에 관한 사항.csv",
#     "VI. 이사회 등 회사의 기관에 관한 사항.csv",
#     "IV. 이사의 경영진단 및 분석의견.csv",
#     "2. 임원의 보수 등.csv",
# ]

high_list = [
    "1. 요약재무정보.csv",
    "2. 연결재무제표.csv",
    "3. 연결재무제표 주석.csv",
    "4. 재무제표.csv",
    "5. 재무제표 주석.csv",
    "3. 자본금 변동사항.csv",
    "6. 배당에 관한 사항 등.csv",
    "IV. 이사의 경영진단 및 분석의견.csv",
]

medium_list = [
"VII. 주주에 관한 사항.csv",
"IX. 계열회사 등에 관한 사항.csv",
"XI. 그 밖에 투자자 보호를 위하여 필요한 사항.csv",


]

#  주주현황, 계열사, 투자자 보호 관련 추가공시, 이사회/임원/의결권 현황 등

###
summary_system = "당신은 금융 전문가입니다. 주어진 금융 보고서 각각에 대해, step-by-step으로 요약하세요."

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import nltk
nltk.download('punkt')

model_dir = "lcw99/t5-base-korean-text-summary"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

max_input_length = 2048

import ast
import os

from sub_func import fin_statement_info # ticker, year, quarter -> df
from sub_func import reports_info # ticker, year, quarter -> df
from sub_func import corp_rel_news_info # ticker, year, start_date, end_date -> df
from sub_func import intl_news_info # year, start_date, end_date -> df
from sub_func import macro_econ_info # year, start_date, end_date -> dict
from sub_func import stock_price_info # ticker, start_date, end_date -> df
from sub_func import index_price_info # sector, start_date, end_date -> df
from sub_func import pattern_info # ticker, date -> df
from sub_func import sector_analysis_info # sector, year, quarter -> dict

from sub_func import get_index_list

from sub_func import to_GPT

from sub_func import preprocess_financial_text

def price_estimation_info():
    return None 

def current_portfolio_info():
    return None

def portfolio_analysis_info():
    return None

# ================================================================================================================ #

def get_ordered_report_dict(ticker, year, quarter):
    """
    중요도에 따라 보고서를 분류한 dict 반환
    """

    df = reports_info(ticker, year, quarter)
    report_importance_dict = {
        'High': {},
        'Medium': {}
    }

    csv_list = list(df.keys())
    for csv in csv_list:
        if csv in high_list:
            report_importance_dict['High'][csv] = ast.literal_eval(df[csv][0])[0]
        elif csv in medium_list:
            report_importance_dict['Medium'][csv] = ast.literal_eval(df[csv][0])[0]
        else:
            pass

    return report_importance_dict

def summarize_text(text):
    """
    금융 관련 문장 요약 모델
    """
    inputs = ["summarize: " + f"{preprocess_financial_text(text)}"]

    inputs = tokenizer(inputs, max_length=max_input_length, truncation=False, return_tensors="pt")
    output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=300)
    decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
    result = nltk.sent_tokenize(decoded_output.strip())[0]

    return result

def sliding_window_summary(text, max_chunk_len=1000, overlap_len=100):
    """
    긴 텍스트를 분할해 요약하고 결과를 재결합
    """
    # 텍스트 분할
    sentences = nltk.sent_tokenize(text)
    chunks = []
    current_chunk = []

    # 슬라이딩 윈도우 방식으로 분할
    for sentence in sentences:
        if len(" ".join(current_chunk + [sentence])) <= max_chunk_len:
            current_chunk.append(sentence)
        else:
            chunks.append(" ".join(current_chunk))
            current_chunk = sentences[
                max(0, len(current_chunk) - overlap_len):]  # 중첩된 부분 유지

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    # 요약 결과 생성
    summaries = [summarize_text(chunk) for chunk in chunks]
    final_summary = " ".join(summaries)
    
    return final_summary

[nltk_data] Downloading package punkt to /Users/yeonsuk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/opt/anaconda3/envs/finTF/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
# ====================================================================================================================== #
TODAY = '20200630'
target_year = '2020'
target_quarter = 'Q2'

quarter_list = ['Q1', 'Q2', 'Q3', 'Q4']

fin_statement_dict = {}
fin_report_dict = {}
corp_rel_news_dict = {}
intl_news_dict = {}
macro_econ_dict = {}
corp_price_dict = {}
index_price_dict = {}
pattern_dict = {}
sector_analysis_dict = {}

# ====================================================================================================================== #

# r_{portfolio manager_{t_5}1}도 불러와야함 | 일단 패스

# ====================================================================================================================== #

# 한 분기 이전 정보 찾기 (ex. 만약 지금이 Q3라면 Q2의 정보를 가져와야 함. 만약 Q1이라면 이전 년도의 Q4를 가져와야 함.)
if target_quarter == 'Q1':
    target_year = f"{int(target_year)-1}"

search_quarter_idx = (quarter_list.index(target_quarter) - 1) % 4
search_quarter = quarter_list[search_quarter_idx]

if search_quarter == 'Q1':
    start_date = target_year + '0101'
    # end_date = target_year + '0331'
    end_date = TODAY
elif search_quarter == 'Q2':
    start_date = target_year + '0401'
    # end_date = target_year + '0630'
    end_date = TODAY
elif search_quarter == 'Q3':
    start_date = target_year + '0701'
    # end_date = target_year + '0930'
    end_date = TODAY
elif search_quarter == 'Q4':
    start_date = target_year + '1001'
    # end_date = target_year + '1231'
    end_date = TODAY

### ====== 티커 필요 없는 항목들 가져오기 ====== ###
# ==== 거시경제 정보 가져오기 ==== #
macro_econ = macro_econ_info(target_year, start_date, end_date)
macro_econ_dict[target_year] = macro_econ

# ==== 국제 뉴스 가져오기 ==== #
intl_news = intl_news_info(target_year, start_date, end_date)
intl_news_dict[f"{start_date}_{end_date}"] = intl_news

# ==== 인덱스 가격 가져오기 ==== #
index_list = get_index_list()
for index in index_list:
    try:
        idx_price = index_price_info(index, start_date, end_date)
        index_price_dict[index] = idx_price

        sector_analysis = sector_analysis_info(index, target_year, search_quarter)
        sector_analysis_dict[index] = sector_analysis
    except:
        continue

# ====================================================================================================================== #

### ====== 티커 기반 항목들 가져오기 ====== ###
tickers_list = os.listdir('../store_data/raw/market_data/price/')
for i in range(len(tickers_list)):

    try:
        target_ticker = tickers_list[i]

        #  ==== 재무제표 분석 정보 가져와서 담아두기 ==== #
        fin_statement = fin_statement_info(target_ticker, target_year, search_quarter)
        fin_statement_dict[target_ticker] = fin_statement

        # ==== 재무보고서 가져와서 담아두기 ==== #
        fin_report = get_ordered_report_dict(target_ticker, target_year, search_quarter)
        report_txt = ""
        for report in fin_report['Medium']:
            report_txt += f"{preprocess_financial_text(fin_report['Medium'][report])}"
        
        # GPT 기반 요약?
        # response = to_GPT(summary_system, report_txt)['choices'][0]['message']['content']
        response = report_txt

        fin_report['Medium'] = ''
        fin_report['Medium'] = response
        fin_report_dict[target_ticker] = fin_report

        # ==== 뉴스 정보 가져오기 ==== #
        corp_rel_news = corp_rel_news_info(target_ticker, target_year, start_date, end_date)
        corp_rel_news_dict[target_ticker] = corp_rel_news

        # ==== 가격 정보 가져오기 ==== #
        corp_price = stock_price_info(target_ticker, start_date, end_date)
        corp_price_dict[target_ticker] = corp_price

        # # ==== 차트 패턴 정보 가져오기 ==== #
        # pattern = pattern_info(target_ticker, end_date)
        # pattern_dict[target_ticker] = pattern

    except:
        continue

재무제표를 불러오는 과정에서 오류가 발생했습니다 | [Errno 2] No such file or directory: '/Users/yeonsuk/investment/finTF/pipeline/sub_func/../../store_data/raw/opendart/store_financial_statement/013700/_013700_재무제표 ().csv'
재무제표를 불러오는 과정에서 오류가 발생했습니다 | [Errno 2] No such file or directory: '/Users/yeonsuk/investment/finTF/pipeline/sub_func/../../store_data/raw/opendart/store_financial_statement/013700/_013700_재무제표 ().csv'
013700의 이전 년도 재무 데이터를 불러올 수 없습니다.
013700의 fin_statement_info 정보를 확인할 수 없습니다.
재무제표를 불러오는 과정에서 오류가 발생했습니다 | [Errno 2] No such file or directory: '/Users/yeonsuk/investment/finTF/pipeline/sub_func/../../store_data/raw/opendart/store_financial_statement/058860'
재무제표를 불러오는 과정에서 오류가 발생했습니다 | [Errno 2] No such file or directory: '/Users/yeonsuk/investment/finTF/pipeline/sub_func/../../store_data/raw/opendart/store_financial_statement/058860'
058860의 이전 년도 재무 데이터를 불러올 수 없습니다.
058860의 fin_statement_info 정보를 확인할 수 없습니다.
재무제표를 불러오는 과정에서 오류가 발생했습니다 | [Errno 2] No such file or directory: '/Users/yeonsuk/

In [5]:
fin_statement_dict
fin_report_dict
corp_rel_news_dict
corp_price_dict
pattern_dict

index_price_dict
sector_analysis_dict

intl_news_dict
macro_econ_dict

,start_date,end_date,pattern,prob
0,2019-01-17,2019-04-30,ascending_wedge,0.987465


In [7]:
sample_dict = {
    'fin_statement': fin_statement_dict['005930'],
    'fin_report': fin_report_dict['005930'],
    'corp_rel_news': corp_rel_news_dict['005930'],
    'corp_price': corp_price_dict['005930'],
    # 'pattern': pattern_dict['005930'],

    'index_price': index_price_dict,
    'sector_analysis': sector_analysis_dict,

    'intl_news': intl_news_dict,
    'macro_econ': macro_econ_dict
}

In [20]:
for key, value in sample_dict.items():
    print(f"Key: {key}, Type: {type(value)}")

import pandas as pd

def convert_to_serializable(obj):
    if isinstance(obj, pd.DataFrame):
        return obj.to_dict(orient="records")
    elif isinstance(obj, dict):
        return {k: convert_to_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_serializable(v) for v in obj]
    else:
        return obj



sample_dict_serializable = convert_to_serializable(sample_dict)

with open("data.json", "w", encoding="utf-8") as f:
    json.dump(sample_dict_serializable, f, ensure_ascii=False, indent=4)


Key: fin_statement, Type: <class 'list'>
Key: fin_report, Type: <class 'dict'>
Key: corp_rel_news, Type: <class 'list'>
Key: corp_price, Type: <class 'list'>
Key: index_price, Type: <class 'dict'>
Key: sector_analysis, Type: <class 'dict'>
Key: intl_news, Type: <class 'dict'>
Key: macro_econ, Type: <class 'dict'>


In [1]:
import pandas as pd

In [4]:
df = pd.read_csv('/Users/yeonsuk/investment/finTF/store_data/raw/opendart/store_reports/000020/2020.09/000020_분기보고서 (2020.09) | 1. 요약재무정보.csv')

In [9]:
list(df['text'])[0].split('\\n')

["['",
 '1. 요약재무정보',
 '가. 요약 연결재무정보',
 '\\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 (단위 : 천원)',
 '\\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0 \\xa0\\xa0 사업연도과\\xa0\\xa0 목',
 '92기 (2020.09.30)',
 '91기 (2019.12.31)',
 '90기 (2018.12.31)',
 '[유동자산]',
 '245,994,161\\xa0',
 '\\xa0235,456,988\\xa0',
 '227,887,463\\xa0',
 '\\xa0 현금및현금성자산',
 '72,397,341\\xa0',
 '\\xa037,779,183\\xa0',
 '40,614,515\\xa0',
 '\\xa0 기타금융자산',
 '64,420,087\\xa0',
 '\\xa0 \\xa0 82,949,068\\xa0',
 '63,444,817\\xa0',
 '\\xa0 매출채권및기타채권',
 '68,897,028\\xa0',
 '\\xa0 \\xa0 79,441,839\\xa0',
 '79,057,839\\xa0',
 '\\xa0 재고자산',
 '38,957,552\\xa0',
 '\\xa0 \\xa0 33,430,792\\xa0',
 '43,717,602\\xa0',
 '\\xa0 기타유동자산',
 '1,208,990\\xa0',
 '\\xa0 \\xa0 \\xa0 1,855,585\\xa0',
 '1,05